In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

import sys
sys.path.append('../script/')
from os.path import exists
import utils
DEVICE = "cuda"
EPOCHS = 100

C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7dadc1502374>", line 9, in <module>
    import utils
  File "../script\utils.py", line 5, in <module>
    from sklearn.model_selection import train_test_split
  File "C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\sklearn\__init__.py", line 80, in <module>
    from .base import clone
  File "C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\sklearn\base.py", line 21, in <module>
    from .utils import _IS_32BIT
  File "C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\sklearn\utils\__init__.py", line 23, in <module>
    from .class_weight import compute_class_weight, compute_sample_weight
  File "C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\sklearn\utils\class_weight.py", line 7, in <module>
    from .validation import _deprecate_positional_a

TypeError: object of type 'NoneType' has no len()

## Add folds No. for CV

In [ ]:
path_fold = "../input/folds/train_folds.csv"
if not exists(path_fold):
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
    df.loc[:, "kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    targets = df.drop("sig_id", axis=1).values

    mskf = MultilabelStratifiedKFold(n_splits=5)
    for fold_, (tr_, va_) in enumerate(mskf.split(X=df, y=targets)):
        df.loc[va_, "kfold"] = fold_
    df.to_csv(path_fold, index=False)

## Params for training function `run_training`

In [ ]:
fold = 0
# optuna 使わないので適当
params = {
    "num_layers": 3,
    "hidden_size": 16,
    "dropout": 0.3,
    "learning_rate": 1e-3,
}
save_model=True

---
## Prototyping training process from HERE
`run_training` function contains the following code blocks:
```python
def run_training(fold, params, save_model=False):
```

In [ ]:
df = pd.read_csv("../input/lish-moa/train_features.csv")
df = utils.process_data(df)
folds = pd.read_csv("../input/folds/train_folds.csv")

## Create aux target 
`nsc_labels` means # of labels found in non-scored train set Which is not available in test set.

In [ ]:
non_scored_df = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
targets_non_scored = non_scored_df.drop("sig_id", axis=1).to_numpy().sum(axis=1)
non_scored_df.loc[:, "nsc_labels"] = targets_non_scored
drop_cols = [c for c in non_scored_df.columns if c not in ("nsc_labels", "sig_id")]
non_scored_df = non_scored_df.drop(drop_cols, axis=1)
folds = folds.merge(non_scored_df, on="sig_id", how="left")
folds.head()

In [6]:
targets = folds.drop(["sig_id", "kfold"], axis=1).columns
features = df.drop("sig_id", axis=1).columns

In [7]:
df = df.merge(folds, on="sig_id", how="left")

In [8]:
print(f'[Fold No.{fold:>3}]\n')
train_df = df[df.kfold != fold].reset_index(drop=True)
valid_df = df[df.kfold == fold].reset_index(drop=True)

[Fold No.  0]



In [9]:
x_tr = train_df[features].to_numpy()
x_va = valid_df[features].to_numpy()

In [10]:
y_tr = train_df[targets].to_numpy()
y_va = valid_df[targets].to_numpy()

In [11]:
dataset_tr = utils.MoaDataset(x_tr, y_tr)
loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=1024, num_workers=8)
dataset_va = utils.MoaDataset(x_va, y_va)
loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=1024, num_workers=8)

In [12]:
#     dataset_te = utils.TestMoADataset(dataset=x_te.iloc[:, 1:].values)
#     loader_te = torch.utils.data.DataLoader(
#         dataset_te, batch_size=1024, num_workers=0, shuffle=False
#     )

In [13]:
model = utils.BaseLine2(
    num_features=x_tr.shape[1],
    num_targets=y_tr.shape[1],
    num_layers=params["num_layers"],
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
)
model.to(DEVICE)

BaseLine2(
  (model): Sequential(
    (0): Linear(in_features=879, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=16, out_features=16, bias=True)
    (7): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=16, out_features=207, bias=True)
  )
)

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=3, threshold=0.00001, mode="min", verbose=True
)
eng = utils.Engine(model, optimizer, device=DEVICE)

In [15]:
loss_best = np.inf
early_stop = 10
early_stop_counter = 0
for ep in range(EPOCHS):
    loss_tr = eng.train(loader_tr)
    loss_va = eng.validate(loader_va)
    scheduler.step(loss_va)
    print(f"epoch={ep:>4}, train_loss={loss_tr:.5f}, valid_loss={loss_va:.5f}, patience={early_stop_counter:>3}/{early_stop:>3}")
    if loss_va < loss_best:
        loss_best = loss_va
        if save_model:
            torch.save(model.state_dict(), f"../weight/model_fold{fold}.bin")
    else:
        early_stop_counter += 1
    if early_stop_counter > early_stop:
        break

fold=0, epoch=0, train_loss=0.7323568682921561, valid_loss=0.6994419813156127
fold=0, epoch=1, train_loss=0.7083098637430292, valid_loss=0.6844298362731933
fold=0, epoch=2, train_loss=0.6890535542839452, valid_loss=0.6666178822517395
fold=0, epoch=3, train_loss=0.6663718098088315, valid_loss=0.6415032505989074
fold=0, epoch=4, train_loss=0.6368942699934307, valid_loss=0.6072230458259582
fold=0, epoch=5, train_loss=0.5969087763836509, valid_loss=0.560306453704834
fold=0, epoch=6, train_loss=0.5456650884527909, valid_loss=0.5043397307395935
fold=0, epoch=7, train_loss=0.48882698228484706, valid_loss=0.442163360118866
fold=0, epoch=8, train_loss=0.42705497616215754, valid_loss=0.38112486600875856
fold=0, epoch=9, train_loss=0.36792014774523285, valid_loss=0.32280845642089845
fold=0, epoch=10, train_loss=0.3124161205793682, valid_loss=0.2702167332172394
fold=0, epoch=11, train_loss=0.26626651930181605, valid_loss=0.225433149933815
fold=0, epoch=12, train_loss=0.22336767849169278, valid_los

In [16]:
print(f'fold={fold}, best vallidation loss={loss_best}')

fold=0, best vallidation loss=0.021769075468182564


```python
return loss_best
```
---

In [20]:
print(f"fold={fold}, best vallidation loss={loss_best}")

fold=0, best vallidation loss=0.021769075468182564


```python
def run_training(fold, params, save_model=False):
    df = pd.read_csv("train_features.csv")
    df = utils.process_data(df)
    folds = pd.read_csv("train_folds.csv")

    # aux target
    non_scored_df = pd.read_csv("train_targets_nonscored.csv")
    targets_non_scored = non_scored_df.drop("sig_id", axis=1).to_numpy().sum(axis=1)
    non_scored_df.loc[:, "nscr"] = targets_non_scored
    drop_cols = [c for c in non_scored_df.columns if c in ("nscr", "sig_id")]
    non_scored_df = non_scored_df.drop(drop_cols, axis=1)
    folds = folds.merge(non_scored_df, on="sig_id", how="left")

    targets = folds.drop(["sig_id", "kfold"], axis=1)
    features = df.drop("sig_id", axis=1).columns

    df = df.merge(folds, on="sig_id", how="left")

    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    x_tr = train_df[features].to_array()
    x_va = valid_df[features].to_array()

    y_tr = train_df[targets].to_array()
    y_va = valid_df[targets].to_array()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=1024, num_workers=8)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=1024, num_workers=8)
    dataset_te = utils.TestMoADataset(dataset=x_te.iloc[:, 1:].values)
    loader_te = torch.utils.data.DataLoader(
        dataset_te, batch_size=1024, num_workers=0, shuffle=False
    )
    model = utils.BaseLine2(
        num_features=x_tr.shape[1],
        num_targets=y_tr.shape[1],
        num_layers=params["num_layers"],
        hidden_size=params["hidden_size"],
        dropout=params["dropout"],
    )
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=3, threshold=0.00001, mode="min", verbose=True
    )
    eng = utils.Engine(model, optimizer, DEVICE)

    loss_best = np.inf
    early_stop = 10
    early_stop_counter = 0
    for ep in range(EPOCHS):
        loss_tr = eng.train(loader_tr)
        loss_va = eng.validate(loader_va)
        scheduler.step(loss_va)
        print(f"fold={fold}, epoch={ep}, train_loss={loss_tr}, valid_loss={loss_va}")
        if loss_va < loss_best:
            loss_best = loss_va
            if save_model:
                torch.save(model.state_dict(), f"model_fold{fold}.bin")
        else:
            early_stop_counter += 1
        if early_stop_counter > early_stop:
            break
```